In [ ]:
import numpy as np
import psana as ps
import matplotlib.pyplot as plt
import sys
import time
import pickle
import os
from Functions import *
from raw_data_class import RawData as RDC
from processed_data_class import processed_data_class as PDC
%matplotlib notebook

In [ ]:
save_dir = '/cds/home/b/bpoult/sample_data/'
calib_dir = '/cds/home/b/bpoult/LCLS_LV_27_Data_Analysis/calibrations/'
runs = np.arange(230,239) # goes from first value in range to last - 1
epix_roi = [80,140,170,330]
xrt_roi = [725,1500]
calibration_file = 'calibration_2'

# Processing or Loading
process_raw = False
on_off = False # Do you want to plot the raw data?
plot_one = 238
plot_one_idx = np.searchsorted(runs,plot_one)
x_axis = 'energy' # 'pixels' or 'energy'
to_cal_file = [calib_dir, calibration_file]

In [ ]:
# Energy Calibration, plot raw calibration runs with pixel axis and populate these arrays
save_cal = False
if save_cal is True:
    calib_runs = list(range(230,238))
    notch_energies = [6.985,6.990,6.995,7.000,7.005,7.010,7.015,7.020] # keV
    xrt_pixels = [1412,1321,1237,1150,1065,980,896,810] # enter 0 if you can't see the notch
    epix_pixels = [0,243,256,269,282,295,0,0] # enter 0 if you can't see the notch

    labels = ['notch_energies','xrt_pixels','epix_pixels','rois','to_cal_file','calib_runs']
    rois = [xrt_roi,epix_roi]
    calib_in = [labels, notch_energies,xrt_pixels,epix_pixels,rois,to_cal_file,calib_runs]
    calib_out = energy_calib.energy_calib(calib_in,save_cal)
    print('')
    print('The loaded calibration settings are: ')
    current_cal = [print(calib_out[0][i] +' : '+ str(calib_out[i+1])) for i in list(range(0,5))+list(range(8,10))]
if os.path.exists(to_cal_file[0]+to_cal_file[1]+'.pkl') and save_cal is False:
    with open(to_cal_file[0] + to_cal_file[1] +'.pkl', "rb") as f:
        calib_out = pickle.load(f)
    print('The loaded calibration settings are: ')
    current_cal = [print(calib_out[0][i] +' : '+ str(calib_out[i+1])) for i in list(range(0,5))+list(range(8,10))]   
    
if not os.path.exists(to_cal_file[0]+to_cal_file[1]+'.pkl'):
    print(to_cal_file[1] + ' does not exist.')
    print('No calibration file loaded.')

In [ ]:
# Raw Data
raw_datas = []
for run in runs:
    scan_name = 'run_' + f'{run:03}'
    ds_string = 'exp=cxix46119:run='+str(run)+':smd'
    print(ds_string)
    if process_raw:
        raw_data = load_data.load_data(save_dir,scan_name,ds_string,epix_roi,xrt_roi)
    else:
        if os.path.exists(save_dir + scan_name + "/" + "rawdata.pkl"):
            with open(save_dir + scan_name + "/" + "rawdata.pkl", "rb") as f:
                raw_data = pickle.load(f)
        else:
            print('You are trying to load a file that does not exist.')
    if os.path.exists(save_dir + scan_name + "/" + "rawdata.pkl"):
        raw_datas = raw_datas + [raw_data]

[load_data.add_cal_info(raw_datas[i],to_cal_file) for i in range(0,len(raw_datas))]
plot_raw.plot_raw(raw_datas, plot_one, x_axis, on_off,to_cal_file)

In [1]:
# Applys an energy window to both spectra. Decide on window based on epix spectrum above.
# Also, reduces the resolution of the xrt spectrometer to match that of the epix.
energy_window = [6.986,7.0158]
for i in range(0,len(raw_datas)):
    reduce_xrt_res.apply_window(raw_datas[i],energy_window)
    reduce_xrt_res.reduce_res(raw_datas[i])


# plt.figure()
# plt.plot(raw_data.epix_energy_windowed,np.sum(raw_data.xrt_red_res,0))
# plt.plot(raw_data.xrt_energy_windowed,np.sum(raw_data.xrt_windowed,0),alpha=0.5)
# plt.show()

NameError: name 'raw_datas' is not defined

In [ ]:
# Inputs for filters
# filters = [filter0,filter1, ...,filterN]

            # filterX[0] = On/Off # True or False
            # filterX[1] = Type # 'bounds','linearity','rms'
            # filterX[2] = [par0,par1,...,parN]
            
                # If Type = 'bounds'    # par0 = 'key_0' # key_X is string from raw_data.getKeys()
                                        # par1 = lower bound # check raw_data.keyX for starting point 
                                        # par2 = upper bound # 'None' if no upper bound
                                        # par3 = number of stds from median allowed
            
                # If Type = 'linearity' # par0 = 'key_0','key_1' # key_0 and key_1 and x and y values, respectively
                                        # par1 = width of filter

                        
                # If Type = 'rms'       # TBD
                
# filterX[3] = [in_1, in_2]
                # in_1 = On/Off # True or false
                # in_2 = plot_one # The run you want to see plots for. Declared above in the 2nd block.

# print(raw_data.getKeys()) 
suspress_output = True
filt0 = [True,'bounds',['low_diode_us',0.285,'None',1.7],[False,plot_one]]
filt1 = [True,'bounds',['high_diode_us',0.26,'None',1.5],[False,plot_one]]
filt2 = [True,'bounds',['photon_energies',0.95,'None',0.75],[False,plot_one]]
filt3 = [True,'bounds',['pulse_energies_fee',0.43,'None',1.5],[False,plot_one]]

# The linearity filters create their fit based on data points that make it through
# the bounds filters.

filt4 = [True,'linearity',['xrt_intensity','epix_intensity',0.08],[True,plot_one]]
filt5 = [True,'linearity',['low_diode_us','epix_intensity',0.1],[True,plot_one]]
filt6 = [True,'linearity',['low_diode_us','xrt_intensity',0.1],[True,plot_one]]

filters = [filt0,filt1,filt2,filt3,filt4,filt5,filt6]
conditions = [Filters.filtering(raw_datas[i],filters,suspress_output) for i in range(0,len(raw_datas))]

In [ ]:
# Make or load processed data
make_pro = True
pro_datas = []
probe_run = [238,make_pro]

for i in range(0,len(raw_datas)):
    if make_pro:
        if i is 0:
            new_scale = raw_datas[np.searchsorted(runs,probe_run[0])].make_pro_data(conditions[np.searchsorted(runs,probe_run[0])],filters)
            probe_run = new_scale.scale_spectrometers(probe_run)
        processed_data = raw_datas[i].make_pro_data(conditions[i],filters)
        pro_datas = pro_datas + [processed_data]
    else:
        if os.path.exists(raw_datas[i].save_dir + raw_datas[i].scan_name + "/" + "pro_data.pkl"):
            with open(raw_datas[i].save_dir + raw_datas[i].scan_name + "/" + "pro_data.pkl", "rb") as f:
                processed_data = pickle.load(f)
                pro_datas = pro_datas + [processed_data]
                print('Loaded processed_data for run ' + processed_data.scan_name)
        else:
            print('You are trying to load a file that does not exist.')
        
    pro_datas[i].scale_spectrometers(probe_run)

In [2]:
total_I_epix = processed_data.epix_intensity
total_I_xrt = processed_data.xrt_intensity
epix = processed_data.epix_windowed
xrt = processed_data.xrt_based
energy = processed_data.epix_energy_windowed
events = processed_data.eventIDs
# Factors of 78: 1, 2, 3, 6, 13, 26, 39, and 78
pairs = [[1,78],[2,39],[3,26],[6,13]]
bin_length = np.int64(pairs[2][0])
number_bins = np.int64(78/bin_length)
bin_array = np.full(number_bins,bin_length)
j = 0
xrt_binned = []
epix_binned = []
for i in bin_array:
    if j is 0:
        left = 0
        right = i
        xrt_binned.append(np.sum(xrt[:,left:right],1))
        epix_binned.append(np.sum(epix[:,left:right],1))
        j = 1
        continue
    left = right
    right = right + i
    xrt_binned.append(np.sum(xrt[:,left:right],1))
    epix_binned.append(np.sum(epix[:,left:right],1))
xrt_binned = np.transpose(np.asarray(xrt_binned))
epix_binned = np.transpose(np.asarray(epix_binned))
normd_xrt = np.divide(xrt_binned,total_I_xrt[:,np.newaxis])
normd_epix = np.divide(epix_binned,total_I_epix[:,np.newaxis])

avg_dist_xrt = np.mean(normd_xrt,0)
avg_dist_epix = np.mean(normd_epix,0)
std_dist_xrt = np.std(normd_xrt,0)
std_dist_epix = np.std(normd_epix,0)

scaling_xrt = 1
scaling_epix = 1
cond_up_xrt = normd_xrt<(avg_dist_xrt[i]+scaling_xrt*std_dist_xrt[i])
cond_down_xrt = normd_xrt>(avg_dist_xrt[i]-scaling_xrt*std_dist_xrt[i])
cond_up_epix = normd_epix<(avg_dist_epix[i]+scaling_epix*std_dist_epix[i])
cond_down_epix = normd_epix>(avg_dist_epix[i]-scaling_epix*std_dist_epix[i])

cond_xrt = (cond_up_xrt & cond_down_xrt)
cond_epix = (cond_up_epix & cond_down_epix)
cond_total = (cond_xrt & cond_epix)

# 1 - 10 scale
rating_xrt = np.sum(cond_xrt,1)/number_bins*10
rating_epix = np.sum(cond_epix,1)/number_bins*10
rating_total = np.sum(cond_total,1)/number_bins*10
avg_rating_xrt = np.round(np.mean(rating_xrt),2)
avg_rating_epix = np.round(np.mean(rating_epix),2)
avg_rating_total = np.round(np.mean(rating_total),2)
std_rating_xrt = np.round(np.std(rating_xrt),2)
std_rating_epix = np.round(np.std(rating_epix),2)
std_rating_total = np.round(np.std(rating_total),2)
# plt.title(filt_param[1]+' '+filt_param[2][0])

min_rating_xrt = 6
min_rating_epix = 6
min_rating_total = 5

good_xrt_shots = events[rating_xrt>min_rating_xrt]
good_epix_shots = events[rating_epix>min_rating_epix]
good_total_shots = events[rating_total>min_rating_total]
print('Good xrt shots: ' + str(good_xrt_shots.shape[0]) + ' | Min Rating: ' + str(min_rating_xrt))
print('Good epix shots: ' + str(good_epix_shots.shape[0]) + ' | Min Rating: ' + str(min_rating_epix))
print('Good total shots: ' + str(good_total_shots.shape[0]) + ' | Min Rating: ' + str(min_rating_total))



plot_ratings = False
if plot_ratings is True:
    plt.figure()
    _ = plt.hist(rating_xrt, 10,rwidth=.4,label='xrt ratings')
    plt.title('Mean: '+ str(avg_rating_xrt) + '+/- ' + str(std_rating_xrt) + ' | Bin Length: ' +str(bin_length) 
              + ' eV | '+ 'Fitler Tightness: ' + str(scaling_xrt))
    plt.legend()
    plt.show()

    plt.figure()
    _ = plt.hist(rating_epix, 10,rwidth=.4,label='epix ratings')
    plt.title('Mean: '+ str(avg_rating_epix) + '+/- ' + str(std_rating_epix) + ' | Bin Length: ' + str(bin_length) 
              + ' eV | '+ 'Fitler Tightness: ' + str(scaling_epix))

    plt.legend()
    plt.show()

    plt.figure()
    plt.title('Mean: '+ str(avg_rating_total) + '+/- ' + str(std_rating_total))
    _ = plt.hist(rating_total, 10,rwidth=.4,label='total rating')
    plt.legend()
    plt.show()

NameError: name 'processed_data' is not defined

In [ ]:
# Plotting the processed data
# plot_pro      .check_SN(processed_data, On/Off): Starts off with randomizing order of shots.
                    # Each data point corresponds to the standard deviation
                    # of the mean of x residuals; epix_spectrum-xrt_spectrum.
                    # Should give us an idea of how signal/noise improves with averaging.
                    # Smoothness of curve could also report on effectiveness of filters.
resid = plot_pro.check_SN(pro_datas[plot_one_idx],True)
